###  Sun Cloud Bridge Condition

Define workspace and project directory.

In [ ]:
import arcpy
import os
from pathlib import Path

path = os.getcwd()
db = os.path.join(path, "Data", "suncloud_bridge.gdb")
arcpy.env.workspace = db
arcpy.Describe(db)


Download the current year's NBI data and save as xlsx file. 

In [ ]:
import pandas as pd
import arcpy
from urllib.parse import urlparse


# url for the NBI data.  Change this to the current year's URL.
url = 'https://www.fhwa.dot.gov/bridge/nbi/2022/delimited/AZ22.txt'

# get the filename
a = urlparse(url)
file_name = os.path.basename(a.path).split('.')[0]
file_name



In [ ]:
# save the data to "Data" folder as xlsx file. 
data_dir = os.path.join(path, "Data")
download_excel = "{0}.xlsx".format(file_name)
out = os.path.join(data_dir, download_excel)


read_file = pd.read_csv(url)
read_file.to_excel (out, index=None)

Covert the Excel sheet into a file geodatabase table. 

In [ ]:
excel_path = out
converted_tb = "{}_tb".format(file_name) # name of the converted table
arcpy.conversion.ExcelToTable(excel_path, converted_tb, "Sheet1", 1, "")

Add LAT and LON fields.  

In [ ]:
# convert lat & lon into DMS coordinates
converted_tb = 'AZ22_tb'
codeblock = """
def to_lon(x):
    x = str(x)
    return('-' + x[0:3] + ' ' + x[3:5] + ' ' + x[5:7] + '.' + x[7:9])
    
def to_lat(x):
    x = str(x)
    return(x[0:2] + ' ' + x[2:4] + ' ' + x[4:6] + '.' + x[6:8])"""

arcpy.management.CalculateField(converted_tb, "LAT", "to_lat(!LAT_016!)", "PYTHON3", codeblock, "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(converted_tb, "LON", "to_lon(!LONG_017!)", "PYTHON3", codeblock, "TEXT", "NO_ENFORCE_DOMAINS")

Convert the table to a feature class

In [ ]:
# using the LAT and LON fields, conver the table to a feature class. 

spatial_ref = arcpy.SpatialReference('WGS 1984')
bridge_fc = "{0}_fc".format(file_name)
out_fc = "{0}\{1}".format(db, bridge_fc)

with arcpy.EnvManager(scratchWorkspace=db,
                      workspace=db):
    arcpy.management.ConvertCoordinateNotation(
        in_table = converted_tb, 
        out_featureclass = out_fc, 
        x_field = "LON", 
        y_field = "LAT", 
        input_coordinate_format="DMS_2",
        output_coordinate_format="DD_2",
        spatial_reference=spatial_ref)

Clean up brige's text fields. 
- Strip white space.
- Remove quotation mark.

In [ ]:

fieldlist=[i.name for i in arcpy.ListFields(bridge_fc) if i.type=='String']
with arcpy.da.UpdateCursor(bridge_fc, fieldlist) as cursor:
    for row in cursor:
        row=[i.strip() if i is not None else None for i in row]
        row=[i.replace("'", "") if i is not None else None for i in row]
        cursor.updateRow(row)

Project the layer.

In [ ]:
def project(in_data, target_sr, out_name):


    in_sr = arcpy.Describe(in_data).spatialReference

    tr = arcpy.ListTransformations (in_sr, target_sr)
    datum_conversion = ''
    if(len(tr)>0):
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]

    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

In [ ]:
target_sr = arcpy.SpatialReference(2223)
project('AZ22_fc', target_sr, 'sun_cloud_bridge_all')

Align the bridge to Sun Cloud road network.
- Snap bridges to the Sun Cloud road network for scoring purpose.


Snap to SC routes w/ 20 meters search distance.

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
gis = GIS('pro')

# save hosted feature layer to db
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

In [ ]:
route_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Routes/FeatureServer/92'
out_name = 'sun_cloud_routes'
save_fl(db, route_url, out_name)

In [ ]:
# backup bridge before snapping
fc = 'sun_cloud_bridge_all'
arcpy.ExportFeatures_conversion(fc, '{}_backup'.format(fc))

In [ ]:
road_network = "sun_cloud_routes" # name of the route layer
snap_config=[road_network, "Edge", "25 Meter"]
arcpy.edit.Snap(fc, [snap_config])

Add type field and attribute culvert or bridge

In [ ]:
arcpy.AddField_management(fc, 'type', 'TEXT')

In [ ]:
# Calculate the total fields.
sql = "CULVERT_COND_062 = 'N'"
selected = arcpy.SelectLayerByAttribute_management(fc, 'NEW_SELECTION', sql)

print(arcpy.GetCount_management(selected)[0])
arcpy.management.CalculateField(
    in_table =selected,
    field="type",
    expression="'Bridge'",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


sql = "CULVERT_COND_062 <> 'N'"
selected = arcpy.SelectLayerByAttribute_management(fc, 'NEW_SELECTION', sql)

print(arcpy.GetCount_management(selected)[0])
arcpy.management.CalculateField(
    in_table =selected,
    field="type",
    expression="'Culvert'",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Decode ownner/maintenance responsibility

MAINTENANCE_021
OWNER_022

In [ ]:
code_desc = {
1 :'State Highway Agency',
2 :'County Highway Agency',
3 :'Town or Township Highway Agency',
4 :'City or Municipal Highway Agency',
11:'State Park, Forest, or Reservation Agency',
12:'Local Park, Forest, or Reservation Agency',
21:'Other State Agencies',
25:'Other Local Agencies',
26:'Private (other than railroad)',
27:'Railroad 31 State Toll Authority',
32:'Local Toll Authority',
60:'Other Federal Agencies (not listed below)',
61:'Indian Tribal Government',
62:'Bureau of Indian Affairs',
63:'Bureau of Fish and Wildlife',
64:'U.S. Forest Service',
66:'National Park Service',
67:'Tennessee Valley Authority',
68:'Bureau of Land Management',
69:'Bureau of Reclamation',
70:'Corps of Engineers (Civil)',
71:'Corps of Engineers (Military)',
72:'Air Force',
73:'Navy/Marines',
74:'Army',
75:'NASA',
76:'Metropolitan Washington Airports Service',
80:'Unknown',
99:'Miscoded data'}

custom_code = """
def get_desc(val):
    return code_desc[val]
"""

In [ ]:
# copy over values to the temp field
fc = 'sun_cloud_bridge_all'
fields = [['_MAINTENANCE_021','TEXT'],['_OWNER_022','TEXT']]
for f in fields:
    _f = f[0]
    print(_f[1:])
    arcpy.management.CalculateField(
        in_table=fc,
        field=_f,
        expression="get_desc(!{}!)".format(_f[1:]),
        expression_type="PYTHON3",
        code_block= custom_code,
        field_type="TEXT"
    )

<!-- Calculate minimum structure rating.  (identical to the LOWEST_RATING) -->

In [ ]:
# # lowest bridge condtion

# fc = 'sun_cloud_bridge_all'
# min_rating = "min(!DECK_COND_058!,!SUPERSTRUCTURE_COND_059!, !SUBSTRUCTURE_COND_060!, !CULVERT_COND_062!)"
# arcpy.management.CalculateField(
#     in_table=fc, 
#     field="val_bser", 
#     expression=min_rating,
#     expression_type="PYTHON3",
#     field_type='LONG')
# field_names = [f.name for f in arcpy.ListFields(fc)]

In [ ]:
# check the current field names
field_names = [f.name for f in arcpy.ListFields(fc)]
field_names

In [ ]:
bridge = 'sun_cloud_bridge_all'    
current_fields = [f.name for f in arcpy.ListFields(bridge) if f.required == False]
# print(current_fields)

target_fields = {
	'structure_number_008': "Structure Number",
	'_maintenance_021': "Maintenance Responsibility",
	'_owner_022': "Owner",
	'structure_len_mt_049': "Structure Length",
	'deck_width_mt_052': "Deck Width",
	'deck_cond_058': "Deck Condition Rating",
	'superstructure_cond_059': "Superstructure Condition Rating",
	'substructure_cond_060': "Substructure Condition Rating",
	'culvert_cond_062': "Culvert Condition Rating",
	'structural_eval_067': "Structural Evaluation Rating",
	'deck_geometry_eval_068': "Deck Geometry Evaluation Rating",
    'bridge_condition':"Bridge Condition",
    'lowest_rating':"Lowest Rating",
	'type':"Type"
}

# delete unused fields
delete_fields = [field for field in current_fields if not field.lower() in target_fields.keys()]
print(delete_fields)
arcpy.management.DeleteField(bridge, delete_fields)


In [ ]:
field_names = [f.name for f in arcpy.ListFields(fc)]
field_names

Covert text fields to numeric, where applicable.  This involves replacing value 'N' to None.

In [ ]:
number_fields = [
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060',
    'CULVERT_COND_062',
    'STRUCTURAL_EVAL_067',
    'DECK_GEOMETRY_EVAL_068'
    ]

In [ ]:
custom_code="""
def attribute(val):
    if val.isnumeric() == True:
        return val
"""
for f in number_fields:
    temp = '_{}'.format(f)
    print (temp)
    arcpy.AddField_management(fc, temp, 'LONG')
    arcpy.management.CalculateField(
    in_table=fc,
    field=temp,
    expression="attribute(!{}!)".format(f),
    expression_type="PYTHON3",
    code_block= custom_code,
    enforce_domains="NO_ENFORCE_DOMAINS"
)



In [ ]:
for f in number_fields:
    arcpy.management.DeleteField(fc, f)

Change field names to lower case.

In [ ]:
current_fields = [f.name for f in arcpy.ListFields(bridge) if f.required == False]

for f in current_fields:
    if f[0]!='_':
        print(f)
        arcpy.AlterField_management(fc, f, '_{}'.format(f))


current_fields = [f.name for f in arcpy.ListFields(bridge) if f.required == False]
current_fields

In [ ]:
target_fields = {
	'_structure_number_008': "Structure Number",
	'_maintenance_021': "Maintenance Responsibility",
	'_owner_022': "Owner",
	'_structure_len_mt_049': "Structure Length",
	'_deck_width_mt_052': "Deck Width",
	'_deck_cond_058': "Deck Condition Rating",
	'_superstructure_cond_059': "Superstructure Condition Rating",
	'_substructure_cond_060': "Substructure Condition Rating",
	'_culvert_cond_062': "Culvert Condition Rating",
	'_structural_eval_067': "Structural Evaluation Rating",
	'_deck_geometry_eval_068': "Deck Geometry Evaluation Rating",
    '_bridge_condition':"Bridge Condition",
    '_lowest_rating':"Lowest Rating",
	'_type':"Structure Type"
}

for item in target_fields:
    new = (item[1:])
    alias = (target_fields[item])
    arcpy.management.AlterField(fc, item, new, alias)